In [1]:
import json
from tqdm import tqdm

In [12]:
big = "memoryalpha"
small = "stexpanded"
top_k = 10

gold_pairs_file = "./data/gold_pairs/" + big + "-" + small + "_formatted.txt"
matched_communities_file = "./results/matched_communities_leiden/top_10_pairs/" + big + "-" + small + "_top_10_pairs.json"

exact_match_file = "./data/exact_match/" + big + "-" + small + "-formatted.json"

communities_big_file = "./results/communities_leiden/" + big + "/final.txt"
communities_small_file = "./results/communities_leiden/" + small + "/final.txt"

small_communities = []
big_communities = []
gold_pairs = []
matched_communities = []

In [13]:
with (open(communities_big_file) as cbf, open(communities_small_file) as csf,
      open(gold_pairs_file) as gpf, open(matched_communities_file) as mcf):
    
    for line in csf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        small_communities.append(numbers_set)

    for line in cbf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        big_communities.append(numbers_set)
    
    for line in gpf:
        numbers_list = [int(num) for num in line.strip().split(";")]
        gold_pairs.append(numbers_list)

    matched_communities_load = json.load(mcf)
    matched_communities = [[int(key), [int(values[i][0]) for i in range(0, top_k)]] for key, values in matched_communities_load.items()]

In [14]:
matched_communities

[[0, [0, 646, 1157, 52, 122, 1103, 46, 524, 950, 606]],
 [1, [648, 1112, 1113, 622, 524, 638, 777, 794, 677, 667]],
 [2, [336, 405, 338, 431, 389, 343, 344, 869, 481, 432]],
 [3, [5, 667, 524, 677, 622, 1157, 46, 52, 373, 950]],
 [4, [759, 429, 761, 758, 427, 567, 425, 432, 473, 507]],
 [5, [427, 759, 761, 817, 714, 788, 81, 689, 461, 688]],
 [6, [507, 473, 759, 758, 1081, 499, 418, 369, 82, 276]],
 [7, [436, 425, 856, 507, 473, 759, 536, 82, 432, 434]],
 [8, [507, 104, 758, 441, 425, 427, 856, 429, 759, 510]],
 [9, [788, 87, 273, 472, 1068, 507, 1073, 33, 1069, 327]],
 [10, [536, 761, 758, 427, 759, 473, 81, 106, 951, 943]],
 [11, [507, 799, 1139, 758, 473, 1128, 599, 510, 882, 425]],
 [12, [761, 536, 759, 757, 1138, 556, 371, 347, 278, 427]],
 [13, [46, 67, 839, 49, 47, 1, 104, 369, 11, 823]],
 [14, [1070, 862, 289, 944, 418, 860, 369, 1080, 428, 602]],
 [15, [104, 1093, 806, 1096, 839, 47, 1, 14, 832, 12]],
 [16, [1069, 1070, 369, 273, 1080, 114, 280, 846, 1088, 1067]],
 [17, [427, 

In [15]:
with open(exact_match_file) as file:
    exact_match = json.load(file)

gold_not_exact = list()
for p in gold_pairs:
    if [p[0], p[1]] not in exact_match:
        gold_not_exact.append([p[0], p[1]])

In [16]:
found = 0
# all_pairs = len(gold_pairs)
all_pairs = len(gold_not_exact)

# for gold_pair in tqdm(gold_pairs):
for gold_pair in tqdm(gold_not_exact):
    for matched_community in matched_communities:
        community_1 = big_communities[matched_community[0]]
        community_2 = set()
        for i in range(0, top_k):
            community_2.update(small_communities[matched_community[1][i]])
        if gold_pair[0] in community_1 and gold_pair[1] in community_2:
            found += 1
            # print(matched_community[0], matched_community[1], gold_pair)
            break
            
print("Gold pairs in matched communities top " + str(top_k) + ": " + str(found / all_pairs * 100) + "%")

100%|██████████| 148/148 [00:04<00:00, 32.14it/s]

Gold pairs in matched communities top 10: 27.7027027027027%


In [6]:
found_big = 0
found_small = 0
for gold_pair in gold_pairs:
    for big_community in big_communities:
        if gold_pair[0] in big_community:
            found_big += 1
            break
    for small_community in small_communities:
        if gold_pair[1] in small_community:
            found_small += 1
            break

print("Big: " + str(found_big / all_pairs * 100) + "%")
print("Small: " + str(found_small / all_pairs * 100) + "%")

Big: 100.0%
Small: 100.0%
